# NLTK: Clasificación de textos con Naive Bayes

* En este notebook vamos a ver como crear un modelo que clasifique las frases según su polaridad (Positiva o Negativa).


* Para ello tenemos 3 conjuntos de frases que son:
    1. **Positivas**: Conjunto de frases clasificadas con polaridad positiva
    2. **Negativas**: Conjunto de frases clasificadas con polaridad negativa
    3. **Test**: Conjunto de frases de test a predecir.
    

* El framework para la clasificación de texto (y para la clasificación en general) es el siguiente:

    - ***Entrenamiento***:
    
        1. Limpieza y ***normalización*** del texto
        2. ***Extracción de características***: esto consiste en crear la estructura de datos necesaria para que la utilice el algoritmo de aprendizaje a usar
        3. ***Algoritmo de Aprendizaje***: Dado el conjunto de características de los texto y la etiqueta (label) de los textos, el algoritmo de aprendizaje dará como resultado un **modelo**.
<br><br>     
    - ***Predicción***:
    
        1. Limpieza y ***normalización*** del texto a clasificar.
        2. ***Extracción de características*** del texto a clasificar. Tanto este punto 2 como el anterior (1) deben de realizarse de la misma manera que en la fase de entrenamiento.
        3. ***Predicción***: Dado el conjunto de características de los ***texto a predecir*** y el **modelo**, este devolverá la ***clasificación*** (o predicción) del texto.
        


<hr>


## Ejemplo con NLTK:


In [ ]:
import re
import nltk
import nltk.classify
from nltk.corpus import stopwords

positive = [('I love this car', 'positive'),
            ('This view is amazing', 'positive'),
            ('I feel great this morning', 'positive'),
            ('I am so excited about the concert', 'positive'),
            ('He is my best friend', 'positive'),
            ('Going well', 'positive'),
            ('Thank you', 'positive'),
            ('Hope you are doing well', 'positive'),
            ('I am very happy', 'positive'),
            ('Good for you', 'positive'),
            ('It is all good. I know about it and I accept it.', 'positive'),
            ('This is really good!', 'positive'),
            ('Tomorrow is going to be fun.', 'positive'),
            ('These are great apples today.', 'positive'),
            ('How about them apples? Thomas is a happy boy.', 'positive'),
            ('I love this sandwich.', 'positive'),
            ('This is an amazing place!', 'positive'),
            ('I feel very good about these beers.', 'positive'),
            ('This is my best work.', 'positive'),
            ('What an awesome view', 'positive')]

negative = [('I do not like this car', 'negative'),
            ('This view is horrible', 'negative'),
            ('I feel tired this morning', 'negative'),
            ('I am not looking forward to the concert', 'negative'),
            ('He is my enemy', 'negative'),
            ('I am a bad boy', 'negative'),
            ('This is not good', 'negative'),
            ('I am bothered by this', 'negative'),
            ('I am not connected with this', 'negative'),
            ('Sadistic creep you ass. Die.', 'negative'),
            ('All sorts of crazy and scary as hell.', 'negative'),
            ('Not his emails, no.', 'negative'),
            ('His father is dead. Returned obviously.', 'negative'),
            ('He has a bomb.', 'negative'),
            ('Too fast to be on foot. We cannot catch them.', 'negative'),
            ('I do not like this restaurant', 'negative'),
            ('I am tired of this stuff.', 'negative'),
            ("I can't deal with this", 'negative'),
            ('He is my sworn enemy!', 'negative'),
            ('My boss is horrible.', 'negative')]

test = [('I feel happy this morning', 'positive'),
        ('Larry is my friend', 'positive'),
        ('I do not like that man', 'negative'),
        ('My house is not great', 'negative'),
        ('Your song is annoying', 'negative'),
        ('The beer was good.', 'positive'),
        ('I do not enjoy my job', 'negative'),
        ("I feel amazing!", 'positive'),
        ('Gary is a friend of mine.', 'positive'),
        ("I can't believe I'm doing this.", 'negative')]

<hr>


## Normalización

* En primer lugar vamos a pasar a normalizar las frases. Para ello realizaremos lo siguiente:

    - Eliminamos los signos de puntuación
    - Eliminamos las Stop-Words
    - Pasamos el texto a minúsculas
    
* Nos creamos una función que realice este procesamiento para las frases dadas.


***NOTA***: *Para este ejemplo en particular se hace una normalización muy básica pero suficiente para realizar este ejemplo con caracter didáctico*.

In [ ]:
def normalize(sentenses):
    """normalizamos la lista de frases"""
    sen = []
    for (words, sentiment) in sentenses:
        words_filtered = []
        for word in words.split():
            word = re.sub(r'[^\w\s]', '', word).lower() # Eliminamos signos de puntuación y lo pasamos a minúsculas
            if len(word) > 2 and word not in stopwords.words(): # Filtramos stop words y las palabras con menos de 3 caracteres
                words_filtered.append(word)
        sen.append((words_filtered, sentiment))
    return sen

<hr>


## Extracción de características

* El algoritmo de aprendizaje (Naive Bayes) necesita una determina estructura de datos de entrada para generar el modelo. Para ello necesitamos crear:

    - Un Diccionario (CUIDADO no es un diccionario python) con todas las palabras del corpus.
    
    - Una tupla que contenga en la primera posición un Booleano si por cada palabra del diccionario aparece o no la palabra de la frase. El segundo elemento de la tupla es la etiqueta de la frase. 
    

* Para ello creamos las siguientes 2 funciones:

In [ ]:
def get_unique_words(sentenses):
    """Función que devuelve una lista con todas las palabras únicas que aparecen en las frases"""
    all_words = []
    for (words, sentiment) in sentenses:
        all_words.extend(words)
    return list(set(all_words))

def extract_features(document):
    """Función que crea el conjunto de entrenamiento del clasificador
       1: Toma todos los documentos del corpus
       2: Toma todas las palabras del corpus
       3: Escribre (True|False) si aparece cada palabra del corpus en la frase
    """
    document_words = set(document)
    features = {}
    for word in unique_words:
        features['contains(%s)' % word] = (word in document_words)
    return features

* A continuación pasamos a crear el conjunto de entrenamiento:

In [ ]:
# Normalizamos las frases
sentenses = normalize(positive+negative)

# Obtenemos las palabras del corpus (diccionario del corpus)
unique_words = get_unique_words(sentenses)

# Construimos el conjunto de entrenamiento
training_set = nltk.classify.apply_features(extract_features, sentenses)

print(training_set[0])
sentenses[0]

<hr>


## Algoritmo de Aprendizaje

* Creamos un objeto de la clase NaiveBayesClassifier y llamamos al método train pasándole los datos de entrenamiento.


* De esta manera ya tenemos el modelo creado para su uso asi como información relativa al modelo.

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

<hr>


## Predicción

* Para la predicción vamos a utilizar el modelo creado para ello vamos a:
    1. Normalizar las frases
    2. Extracción de características de la frase
    3. Predicción

* Veamos a continuación como hacerlo y como clasificamos:

In [ ]:
test = normalize(test)   # Normalizamos
for sen in test:
    sentense_features = extract_features(sen[0])   # Extracción de características
    polarity = classifier.classify(sentense_features)   # Clasificación
    print('Frase: ' + str(sen[0]))
    print('\tPolaridad: ' + sen[1] + ' - Predicción: ' + polarity + ' -> ' + ('BIEN' if polarity==sen[1] else 'MAL'))

<hr>


# Modelo

* Veamos a continuación algunas características del modelo:


* Podemo ver por ejemplo la probabilidad de que una frase pertenezca a una etiqueta (label):

In [ ]:
prob_positive = classifier._label_probdist.prob('positive')
prob_negative =  classifier._label_probdist.prob('negative')
print ('-> P(positivo) = ' + str(prob_positive))
print ('-> P(negativo) = ' + str(prob_negative))

* Otra cosa que podemos ver es como de relevante son las palabras que se han utilizado para construir el modelo en función de la etiqueta.


* Vemos por ejemplo que si la palabra "good" aparece en una frase (*contains(good) = True*), esta frase tiene 3 veces más probabilidades de clasificarse como positiva que como negativa. Esta cálculo (según el código) lo hacen de la siguiente manera:


$$positi : negati = \frac{P(good|positivo)}{P(good|negativo)}$$

$$negati : positi = \frac{P(good|negativo)}{P(good|positivo)}$$

In [ ]:
classifier.show_most_informative_features(5)

* Podemos ver tambien como calcula el peso para determinar si una frase es positiva o negativa:

In [ ]:
# Ejemplo POSITIVO
t = "I feel happy this morning"
print ('Predicción: ' + classifier.classify(extract_features(t.split())))
print (classifier.prob_classify(extract_features(t.split()))._prob_dict)

In [ ]:
# Ejemplo NEGATIVO
t = "I do not like that man"
print ('Predicción: ' + classifier.classify(extract_features(t.split())))
print (classifier.prob_classify(extract_features(t.split()))._prob_dict)

* Por último podemos ver la probabilidad de que una frase se clasifique como positiva o negativa en función de que aparezca o no una determinada palabra:

In [ ]:
print("Probabilidad de que la frase sea NEGATIVA si APARECE la palabra 'good':")
print (classifier._feature_probdist[('negative', 'contains(good)')].prob(True))

print("\nProbabilidad de que la frase sea POSITIVA si APARECE la palabra 'good':")
print (classifier._feature_probdist[('positive', 'contains(good)')].prob(True))

print("\nProbabilidad de que la frase sea NEGATIVA si NO APARECE la palabra 'good':")
print (classifier._feature_probdist[('negative', 'contains(good)')].prob(False))

print("\nProbabilidad de que la frase sea POSITIVA si NO APARECE la palabra 'good':")
print (classifier._feature_probdist[('positive', 'contains(good)')].prob(False))